In [ ]:
from os import environ

from autogen import ConversableAgent, run
from autogen.messages.run_events import InputRequestEvent
from autogen.tools import tool

llm_config = {"config_list": [{"api_key": environ["OPENAI_API_KEY"], "model": "gpt-4o-mini"}]}

# Single agent run

In [ ]:
@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


agent = ConversableAgent(name="Alice", llm_config=llm_config, human_input_mode="TERMINATE")

response = run(
    agent,
    initial_message="What is the meaning of life? Call the appropriate function to find out",
    tools=[get_meaning_of_life],
)


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Two agent run

In [ ]:
agent_1 = ConversableAgent(name="Alice", llm_config=llm_config)
agent_2 = ConversableAgent(name="Bob", llm_config=llm_config)

response = run(agent_1, agent_2, initial_message="What is the meaning of life?")


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Multi agent run with chat manager

In [ ]:
import multiprocessing

multiprocessing.set_start_method("spawn", force=True)


def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


planner = ConversableAgent(
    name="Planner",
    system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    llm_config=llm_config,
)

reviewer = ConversableAgent(
    name="Reviewer",
    system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    llm_config=llm_config,
)

teacher = ConversableAgent(
    name="Teacher",
    system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    llm_config=llm_config,
)

teacher.register_for_llm()(submit_plan)
planner.register_for_execution()(submit_plan)
reviewer.register_for_execution()(submit_plan)

# # Set up the Group Chat Manager with a termination condition (when teacher says DONE!)
# chat_manager = GroupChatManager(terminate_on=Keyword("DONE!"), llm_config=llm_config)

# Run the workflow with the three agents and the group chat manager
response = run(
    planner,
    reviewer,
    teacher,
    initial_message="Create lesson plans for 4th grade.",
    # chat_manager=chat_manager,
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)